# **Jacobian Saliency Map**

# Imports

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize

In [6]:
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

# Data

In [7]:
(X_train, y_train), (X_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

In [8]:
X_train = np.transpose(X_train, (0, 3, 1, 2)).astype(np.float32)
X_test = np.transpose(X_test, (0, 3, 1, 2)).astype(np.float32)

# Model

In [9]:
class MNIST_NN(nn.Module):
    def __init__(self):
        super(MNIST_NN, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
        self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
        self.fc_2 = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 10)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x

In [10]:
def train(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(min_pixel_value, max_pixel_value),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(1, 28, 28),
        nb_classes=10,
    )
    
    start_time = time.time()
    classifier.fit(X_train, y_train, batch_size=64, nb_epochs=5)
    
    # Prediction on normal X_test samples
    predictions_benign = classifier.predict(X_test)
    accuracy = np.sum(np.argmax(predictions_benign, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on benign test examples: {}%".format(accuracy * 100))
    
    # Prediction on perturbated samples
    attack = SaliencyMapMethod(classifier=classifier)
    X_test_adv = attack.generate(x=X_test)
    
    predictions_adv = classifier.predict(X_test_adv)
    accuracy = np.sum(np.argmax(predictions_adv, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
    
    # Time
    training_time = time.time() - start_time
    print(f"Time: {training_time:.2f} seconds")
    
    predictions_benign_proba = F.softmax(torch.tensor(predictions_benign), dim=1).numpy()
    predictions_adv_proba = F.softmax(torch.tensor(predictions_adv), dim=1).numpy()
    
    return predictions_benign, predictions_adv, predictions_benign_proba, predictions_adv_proba

In [11]:
model = MNIST_NN()
predictions_benign, predictions_adv, predictions_benign_proba, predictions_adv_proba = train(model)

Accuracy on benign test examples: 98.25%


JSMA:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy on adversarial test examples: 0.36%
Time: 45.81 seconds


# Evaluations

## Helpers

In [12]:
def calculate_weighted_rates(cm):
    num_classes = cm.shape[0]
    total_instances = np.sum(cm)
    
    weighted_TPR = 0
    weighted_TNR = 0
    weighted_FPR = 0
    weighted_FNR = 0
    
    for class_index in range(num_classes):
        TP = cm[class_index, class_index]
        FP = cm[:, class_index].sum() - TP
        FN = cm[class_index, :].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        
        class_size = cm[class_index, :].sum()  # Total true instances for the class
        
        # Calculate rates
        TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
        TNR = TN / (TN + FP) if (TN + FP) != 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) != 0 else 0
        FNR = FN / (FN + TP) if (FN + TP) != 0 else 0
        
        # Weight by class proportion and accumulate
        weighted_TPR += (class_size / total_instances) * TPR
        weighted_TNR += (class_size / total_instances) * TNR
        weighted_FPR += (class_size / total_instances) * FPR
        weighted_FNR += (class_size / total_instances) * FNR

    return weighted_TPR, weighted_TNR, weighted_FPR, weighted_FNR

In [13]:
def calculate_class_metrics_macro(cm, class_index):

    TP = cm[class_index, class_index]
    FP = cm[:, class_index].sum() - TP
    FN = cm[class_index, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
    TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
    FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
    FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
    
    return TPR, TNR, FPR, FNR

In [14]:
def calculate_metrics_micro(cm):
    
    TP_sum = np.sum([cm[i, i] for i in range(cm.shape[0])])  # Sum of True Positives
    FP_sum = np.sum(cm) - np.sum(np.diag(cm))  # Sum of False Positives (total minus TP)
    FN_sum = FP_sum  # In micro-averaging for multi-class, FN and FP are equivalent in sum
    TN_sum = np.sum(cm) * (cm.shape[0] - 1) - 2 * FP_sum  # Adjusting TN for multi-class
    
    TPR_micro = TP_sum / (TP_sum + FN_sum) if (TP_sum + FN_sum) != 0 else 0  
    TNR_micro = TN_sum / (TN_sum + FP_sum) if (TN_sum + FP_sum) != 0 else 0  
    FPR_micro = FP_sum / (FP_sum + TN_sum) if (FP_sum + TN_sum) != 0 else 0 
    FNR_micro = FN_sum / (FN_sum + TP_sum) if (FN_sum + TP_sum) != 0 else 0  
    
    return TPR_micro, TNR_micro, FPR_micro, FNR_micro

## Weighted, Macro, Micro

In [15]:
def metrics_weighted(predictions_benign, predictions_adv):
    
    # Reshape
    y_true = np.argmax(y_test, axis=1)
    y_pred_benign = np.argmax(predictions_benign, axis=1)
    y_pred_adv = np.argmax(predictions_adv, axis=1)
    
    # Acc, Prec, Rec, F1
    precision_benign, recall_benign, f1_benign, _ = precision_recall_fscore_support(y_true, y_pred_benign, average='weighted')
    accuracy_benign = accuracy_score(y_true, y_pred_benign)

    precision_adv, recall_adv, f1_adv, _ = precision_recall_fscore_support(y_true, y_pred_adv, average='weighted')
    accuracy_adv = accuracy_score(y_true, y_pred_adv)
    
    # TPR, TNR, FPR, FNR
    cm_benign = confusion_matrix(y_true, y_pred_benign)
    cm_adv = confusion_matrix(y_true, y_pred_adv)
    
    TPR_benign, TNR_benign, FPR_benign, FNR_benign = calculate_weighted_rates(cm_benign)
    TPR_adv, TNR_adv, FPR_adv, FNR_adv = calculate_weighted_rates(cm_adv)
    
    # AUC
    auc_benign = roc_auc_score(y_test, predictions_benign_proba, multi_class='ovr', average='weighted')
    auc_adv = roc_auc_score(y_test, predictions_adv_proba, multi_class='ovr', average='weighted')
    
    print("Weighted")
    print(f"Benign Acc: {accuracy_benign:.4f}")
    print(f"Benign Prec: {precision_benign:.4f}")
    print(f"Benign Rec: {recall_benign:.4f}")
    print(f"Benign F1: {f1_benign:.4f}")
    
    print(f"Adv Acc: {accuracy_adv:.4f}")
    print(f"Adv Prec: {precision_adv:.4f}")
    print(f"Adv Rec: {recall_adv:.4f}")
    print(f"Adv F1: {f1_adv:.4f}")
    
    print(f"Benign TPR: {TPR_benign:.4f}") 
    print(f"Benign TNR: {TNR_benign:.4f}") 
    print(f"Benign FPR: {FPR_benign:.4f}")
    print(f"Benign FNR: {FNR_benign:.4f}")
    
    print(f"Adv TPR: {TPR_adv:.4f}") 
    print(f"Adv TNR: {TNR_adv:.4f}") 
    print(f"Adv FPR: {FPR_adv:.4f}")
    print(f"Adv FNR: {FNR_adv:.4f}")
    
    print(f"Benign AUC: {auc_benign:.4f}")
    print(f"Adv AUC: {auc_adv:.4f}")

In [16]:
metrics_weighted(predictions_benign, predictions_adv)

Weighted
Benign Acc: 0.9825
Benign Prec: 0.9826
Benign Rec: 0.9825
Benign F1: 0.9825
Adv Acc: 0.0036
Adv Prec: 0.0017
Adv Rec: 0.0036
Adv F1: 0.0023
Benign TPR: 0.9825
Benign TNR: 0.9980
Benign FPR: 0.0020
Benign FNR: 0.0175
Adv TPR: 0.0036
Adv TNR: 0.8933
Adv FPR: 0.1067
Adv FNR: 0.9964
Benign AUC: 0.9997
Adv AUC: 0.8538


/Users/elvin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
def metrics_macro():
    y_true = np.argmax(y_test, axis=1)
    y_pred_benign = np.argmax(predictions_benign, axis=1)
    y_pred_adv = np.argmax(predictions_adv, axis=1)

    
    # Acc, Prec, Rec, F1
    precision_benign, recall_benign, f1_benign, _ = precision_recall_fscore_support(y_true, y_pred_benign, average='macro')
    accuracy_benign = accuracy_score(y_true, y_pred_benign)
    
    precision_adv, recall_adv, f1_adv, _ = precision_recall_fscore_support(y_true, y_pred_adv, average='macro')
    accuracy_adv = accuracy_score(y_true, y_pred_adv)
    
    cm_benign = confusion_matrix(y_true, y_pred_benign)
    cm_adv = confusion_matrix(y_true, y_pred_adv)
    
    # TPR, TNR, FPR, FNR
    TPR_benign, TNR_benign, FPR_benign, FNR_benign = np.mean([calculate_class_metrics_macro(cm_benign, i) for i in range(10)], axis=0)
    TPR_adv, TNR_adv, FPR_adv, FNR_adv = np.mean([calculate_class_metrics_macro(cm_adv, i) for i in range(10)], axis=0)
    
    # AUC
    auc_benign = roc_auc_score(y_test, predictions_benign_proba, multi_class='ovr', average='macro')
    auc_adv = roc_auc_score(y_test, predictions_adv_proba, multi_class='ovr', average='macro')
    
    print("Macro")
    print(f"Benign Acc: {accuracy_benign:.4f}")
    print(f"Benign Prec: {precision_benign:.4f}")
    print(f"Benign Rec: {recall_benign:.4f}")
    print(f"Benign F1: {f1_benign:.4f}")
    
    print(f"Adv Acc: {accuracy_adv:.4f}")
    print(f"Adv Prec: {precision_adv:.4f}")
    print(f"Adv Rec: {recall_adv:.4f}")
    print(f"Adv F1: {f1_adv:.4f}")
    
    print(f"Benign TPR: {TPR_benign:.4f}") 
    print(f"Benign TNR: {TNR_benign:.4f}") 
    print(f"Benign FPR: {FPR_benign:.4f}")
    print(f"Benign FNR: {FNR_benign:.4f}")
    
    print(f"Adv TPR: {TPR_adv:.4f}") 
    print(f"Adv TNR: {TNR_adv:.4f}") 
    print(f"Adv FPR: {FPR_adv:.4f}")
    print(f"Adv FNR: {FNR_adv:.4f}")
    
    print(f"Benign AUC: {auc_benign:.4f}")
    print(f"Adv AUC: {auc_adv:.4f}")

In [18]:
metrics_macro()

Macro
Benign Acc: 0.9825
Benign Prec: 0.9827
Benign Rec: 0.9821
Benign F1: 0.9823
Adv Acc: 0.0036
Adv Prec: 0.0018
Adv Rec: 0.0039
Adv F1: 0.0025
Benign TPR: 0.9821
Benign TNR: 0.9981
Benign FPR: 0.0019
Benign FNR: 0.0179
Adv TPR: 0.0039
Adv TNR: 0.8897
Adv FPR: 0.1103
Adv FNR: 0.9961
Benign AUC: 0.9997
Adv AUC: 0.8494


/Users/elvin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
def metrics_micro():
    y_true = np.argmax(y_test, axis=1)
    y_pred_benign = np.argmax(predictions_benign, axis=1)
    y_pred_adv = np.argmax(predictions_adv, axis=1)

    # Acc, Prec, Rec, F1
    precision_benign, recall_benign, f1_benign, _ = precision_recall_fscore_support(y_true, y_pred_benign, average='micro')
    accuracy_benign = accuracy_score(y_true, y_pred_benign)
    
    precision_adv, recall_adv, f1_adv, _ = precision_recall_fscore_support(y_true, y_pred_adv, average='micro')
    accuracy_adv = accuracy_score(y_true, y_pred_adv)
    
    # TPR, TNR, FPR, FNR
    cm_benign = confusion_matrix(y_true, y_pred_benign)
    cm_adv = confusion_matrix(y_true, y_pred_adv)
    
    TPR_benign, TNR_benign, FPR_benign, FNR_benign = calculate_metrics_micro(cm_benign)
    TPR_adv, TNR_adv, FPR_adv, FNR_adv = calculate_metrics_micro(cm_adv)
    
    # AUC
    auc_benign = roc_auc_score(y_test, predictions_benign_proba, multi_class='ovr', average='micro')
    auc_adv = roc_auc_score(y_test, predictions_adv_proba, multi_class='ovr', average='micro')
    
    print("Micro")
    print(f"Benign Acc: {accuracy_benign:.4f}")
    print(f"Benign Prec: {precision_benign:.4f}")
    print(f"Benign Rec: {recall_benign:.4f}")
    print(f"Benign F1: {f1_benign:.4f}")
    
    print(f"Adv Acc: {accuracy_adv:.4f}")
    print(f"Adv Prec: {precision_adv:.4f}")
    print(f"Adv Rec: {recall_adv:.4f}")
    print(f"Adv F1: {f1_adv:.4f}")
    
    print(f"Benign TPR: {TPR_benign:.4f}") 
    print(f"Benign TNR: {TNR_benign:.4f}") 
    print(f"Benign FPR: {FPR_benign:.4f}")
    print(f"Benign FNR: {FNR_benign:.4f}")
    
    print(f"Adv TPR: {TPR_adv:.4f}") 
    print(f"Adv TNR: {TNR_adv:.4f}") 
    print(f"Adv FPR: {FPR_adv:.4f}")
    print(f"Adv FNR: {FNR_adv:.4f}")
    
    print(f"Benign AUC: {auc_benign:.4f}")
    print(f"Adv AUC: {auc_adv:.4f}")

In [20]:
metrics_micro()

Weighted
Benign Acc: 0.9825
Benign Prec: 0.9825
Benign Rec: 0.9825
Benign F1: 0.9825
Adv Acc: 0.0036
Adv Prec: 0.0036
Adv Rec: 0.0036
Adv F1: 0.0036
Benign TPR: 0.9825
Benign TNR: 0.9981
Benign FPR: 0.0019
Benign FNR: 0.0175
Adv TPR: 0.0036
Adv TNR: 0.8755
Adv FPR: 0.1245
Adv FNR: 0.9964
Benign AUC: 0.9997
Adv AUC: 0.8423
